In [1]:
%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
#from zlib import crc32

In [2]:
#rm -rf ./data/housing.csv
#!wget -P ./data https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv

In [3]:
housing = pd.read_csv('./data/housing.csv')

In [4]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [5]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [6]:
housing["ocean_proximity"].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [7]:
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [8]:
ifig=1;plt.close(ifig)
housing.hist(bins=50, figsize=(10,10));
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
def test_set_check(identifier, test_ratio):
    return(crc32(np.int64(identifier) &0xffffffff < test_ratio * 2**32))

def split_train_test_set_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return(data.loc[~in_test_set], data.loc[in_test_set])
    

In [10]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state = 42)

In [11]:
len(train_set), len(test_set)

(16512, 4128)

**Use income to stratify the sampling**

In [12]:
housing['income_cat'] = pd.cut(housing['median_income'], bins = [0,1.5,3,4.5,6,np.inf], labels = [1,2,3,4,5])

In [13]:
housing['income_cat'] 

0        5
1        5
2        5
3        4
4        3
        ..
20635    2
20636    2
20637    2
20638    2
20639    2
Name: income_cat, Length: 20640, dtype: category
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

In [14]:
ifig=2;plt.close(ifig);plt.figure(ifig)  
housing['income_cat'].hist();
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
from sklearn.model_selection import StratifiedShuffleSplit
mysplit = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)

In [16]:
# mysplit.split is a generator and has to be used in a loop
for train_index, test_index in mysplit.split(housing, housing['income_cat']):  
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [17]:
#StratifiedShuffleSplit?

In [18]:
strat_test_set['income_cat'].value_counts()/len(strat_test_set)

3    0.350533
2    0.318798
4    0.176357
5    0.114583
1    0.039729
Name: income_cat, dtype: float64

In [19]:
housing['income_cat'].value_counts()/len(housing)

3    0.350581
2    0.318847
4    0.176308
5    0.114438
1    0.039826
Name: income_cat, dtype: float64

In [20]:
for df in (strat_train_set, strat_test_set):  #not sure why this is done says "return data to original form" but why? 
    df.drop("income_cat", axis = 1, inplace = True) #just good practice?

In [21]:
housing = strat_train_set.copy()

# Data visualization

In [22]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
ifig=3;plt.close(ifig);fig = plt.figure(ifig, figsize=(8,8))

states = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')

ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent([-125, -110, 30, 45], crs=ccrs.PlateCarree())
#fig = plt.figure(figsize=(8, 8))ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(states, edgecolor='gray')

plt.scatter(housing['longitude'],housing['latitude'], alpha =0.1, c = housing["median_house_value"], cmap = 'jet', s=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
ifig=3;plt.close(ifig);plt.figure(ifig)  

plt.hexbin(housing['longitude'], housing['latitude'], housing['median_income'], bins = 15, cmap = 'afmhot')


plt.colorbar()#label=r'$\log_{10}({\rm population})$')
plt.clim(3, 7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
#plt.hexbin?

In [25]:
ifig=4;plt.close(ifig);plt.figure(ifig)  
plt.hist2d(housing['longitude'], housing['latitude'], density = True,
           bins = 125, cmap = 'afmhot');
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
corr_matrix = housing.corr()

In [27]:
corr_matrix['median_house_value'].sort_values(ascending=False)

median_house_value    1.000000
median_income         0.687160
total_rooms           0.135097
housing_median_age    0.114110
households            0.064506
total_bedrooms        0.047689
population           -0.026920
longitude            -0.047432
latitude             -0.142724
Name: median_house_value, dtype: float64

In [28]:
from pandas.plotting import scatter_matrix
attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
scatter_matrix(housing[attributes], figsize=(12,8));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
housing['rooms_per_household'] = housing['total_rooms']/housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms']/housing['total_rooms']
housing['population_per_household'] = housing['population']/housing['households']

In [30]:
corr_matrix2 = housing.corr()
corr_matrix2['median_house_value'].sort_values(ascending=False)

median_house_value          1.000000
median_income               0.687160
rooms_per_household         0.146285
total_rooms                 0.135097
housing_median_age          0.114110
households                  0.064506
total_bedrooms              0.047689
population_per_household   -0.021985
population                 -0.026920
longitude                  -0.047432
latitude                   -0.142724
bedrooms_per_room          -0.259984
Name: median_house_value, dtype: float64

In [31]:
housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [33]:
housing_cat = housing[['ocean_proximity']]
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

In [34]:
imputer = SimpleImputer(strategy='median')

In [35]:
housing_num = housing.drop('ocean_proximity', axis=1)
imputer.fit(housing_num)
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

In [36]:
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self,X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        

In [37]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [38]:
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")), #fills NaNs with median
                        ('attribs_adder', CombinedAttributesAdder()), #
                        ('std_scaler', StandardScaler())])

In [39]:
from sklearn.compose import ColumnTransformer
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([('num', num_pipeline, num_attribs),
                                  ('cat', OneHotEncoder(), cat_attribs)])

In [40]:
housing_prepared = full_pipeline.fit_transform(housing)

In [41]:
housing_prepared

array([[-1.15604281,  0.77194962,  0.74333089, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.18684903, -1.34218285,  0.18664186, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.        ,
         0.        ,  0.        ],
       [-1.43579109,  0.99645926,  1.85670895, ...,  0.        ,
         1.        ,  0.        ]])

# Training

In [42]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared,housing_labels)

LinearRegression()

In [43]:
some_data = housing.iloc[:5]
some_lables = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

In [44]:
print('Predictions:' ,lin_reg.predict(some_data_prepared))
print('Labels:' , list(some_lables))

Predictions: [210644.60459286 317768.80697211 210956.43331178  59218.98886849
 189747.55849879]
Labels: [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


In [45]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels,housing_predictions)
lin_rmse = np.sqrt(lin_mse)

In [46]:
lin_rmse #not so good if typical vals are 120-260000

68628.19819848923

In [47]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor()

In [48]:
housing_predictions = tree_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels,housing_predictions)
lin_rmse = np.sqrt(lin_mse)

In [49]:
lin_rmse  #overfitting

0.0

In [50]:
housing_labels

17606    286600.0
18632    340600.0
14650    196900.0
3230      46300.0
3555     254500.0
           ...   
6563     240200.0
12053    113000.0
13908     97800.0
11159    225900.0
15775    500001.0
Name: median_house_value, Length: 16512, dtype: float64

# Cross Validation

In [51]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", cv=10)

In [52]:
tree_rmse_scores = np.sqrt(-scores)

In [53]:
print(tree_rmse_scores, tree_rmse_scores.mean(), tree_rmse_scores.std())

[69012.70637059 67987.05298817 70300.49724873 67313.34587981
 71996.48683668 74245.29096317 71309.73043812 71103.28417959
 77912.93007484 70468.48792011] 71164.98128997863 2941.7397477362824


In [54]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor()

In [55]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels,housing_predictions)
forest_rmse = np.sqrt(forest_mse)

In [56]:
forest_rmse

18634.70871103129

In [57]:
scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", cv=10)

In [58]:
forest_rmse_scores = np.sqrt(-scores)
print(forest_rmse_scores, forest_rmse_scores.mean(), forest_rmse_scores.std())

[49613.70863575 47601.57598391 49914.63161855 52363.26375882
 49703.1504364  53359.54259254 48995.78091627 47962.54345273
 52997.72900616 50152.25686473] 50266.418326584906 1905.9923262658504


In [59]:
pickle.dump(housing_prepared, open('./housing_prepared', 'wb'))

In [60]:
# Grid search
from sklearn.model_selection import GridSearchCV

param_grid = [{'n_estimators': [30,40,50], 'max_features':[8,12,14,16]},\
              {'bootstrap': [False], 'n_estimators':[3,10,100], 'max_features':[2,3,4]}]



In [61]:
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, \
                        scoring = 'neg_mean_squared_error', return_train_score=True)


In [65]:
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [8, 12, 14, 16],
                          'n_estimators': [30, 40, 50]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10, 100]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [66]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 4, 'n_estimators': 100}

In [67]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, max_features=4)

In [68]:
cvres = grid_search.cv_results_

In [69]:
for mean_score,params in zip(cvres['mean_test_score'],cvres['params']):
    print(np.sqrt(-mean_score), params)

50243.93706403439 {'max_features': 8, 'n_estimators': 30}
49881.75538161339 {'max_features': 8, 'n_estimators': 40}
49630.99453434051 {'max_features': 8, 'n_estimators': 50}
50546.42633770562 {'max_features': 12, 'n_estimators': 30}
50157.45817707806 {'max_features': 12, 'n_estimators': 40}
50142.94819077683 {'max_features': 12, 'n_estimators': 50}
50867.897995616106 {'max_features': 14, 'n_estimators': 30}
50554.76374515363 {'max_features': 14, 'n_estimators': 40}
50375.13110634346 {'max_features': 14, 'n_estimators': 50}
51049.20931235796 {'max_features': 16, 'n_estimators': 30}
50870.562640514916 {'max_features': 16, 'n_estimators': 40}
50813.244786135394 {'max_features': 16, 'n_estimators': 50}
62523.63185159802 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54578.40232334915 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
50830.24260414107 {'bootstrap': False, 'max_features': 2, 'n_estimators': 100}
59969.698510837516 {'bootstrap': False, 'max_features': 3,

In [70]:
featue_importances = grid_search.best_estimator_.feature_importances_

In [73]:
grid_search.best_estimator_.feature_importances_?

In [71]:
featue_importances

array([8.71936876e-02, 7.39010190e-02, 4.16864035e-02, 2.34026108e-02,
       1.91878738e-02, 2.22512192e-02, 1.92494516e-02, 2.82396980e-01,
       7.23952752e-02, 9.75306902e-02, 8.05587405e-02, 1.85668745e-02,
       1.47559030e-01, 6.73035089e-05, 5.87833487e-03, 8.17450619e-03])

In [74]:
extra_attribs = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_hhold']  
cat_encoder_attribs = list(full_pipeline.named_transformers_['cat'].categories_[0])
attributes = num_attribs + extra_attribs + cat_encoder_attribs #order they were done in pipeline

In [77]:
sorted(zip(featue_importances, attributes), reverse=True)

[(0.2823969798777568, 'median_income'),
 (0.14755902963205725, 'INLAND'),
 (0.09753069016963604, 'pop_per_hhold'),
 (0.08719368755418679, 'longitude'),
 (0.08055874052013472, 'bedrooms_per_hhold'),
 (0.07390101904691991, 'latitude'),
 (0.07239527523378579, 'rooms_per_hhold'),
 (0.04168640353084164, 'housing_median_age'),
 (0.023402610825583404, 'total_rooms'),
 (0.02225121918373635, 'population'),
 (0.01924945157969725, 'households'),
 (0.019187873822976466, 'total_bedrooms'),
 (0.018566874459470303, '<1H OCEAN'),
 (0.008174506185466801, 'NEAR OCEAN'),
 (0.005878334868850026, 'NEAR BAY'),
 (6.730350890040917e-05, 'ISLAND')]

# Evaluate on the test set

In [81]:
final_model = grid_search.best_estimator_
X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value'].copy()

In [82]:
X_test_prepared = full_pipeline.transform(X_test)

In [91]:
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [92]:
final_rmse

46543.52415352989

In [85]:
from scipy import stats

In [93]:
confidence = 0.99
squared_errors = (final_predictions- y_test)**2
np.sqrt(stats.t.interval(confidence, len(squared_errors) -1, loc=squared_errors.mean(), scale=stats.sem(squared_errors)))

array([43958.96456416, 48991.92500512])